# ETL Report
This Notebook serves as the ETL report for Group 2's assessment using the United States Census Bureau Annual Business Survey (ABS) APIs. The steps below are outlined to extract, transform, and load the four APIs for analysis.

There are a total of four transformation processes, with one for each group member. Each process is noted with the individual group member's name.

Before starting the processes, you will need to register an API key to access the datasets. It can be obtained by following the numbered directions at this link: https://www.census.gov/data/developers/guidance/api-user-guide.Help_&_Contact_Us.html

# Data Extraction

In [80]:
# Run the following packages and libraries:
import requests
import json
import pandas as pd
from pandas import json_normalize

Once you have your API key, you will need to replace YOUR_API_KEY in each of the URLs below with your API key.

When you run the four cells below, it will extract the data from the each of the APIs and read the data from a JSON into a dataframe using pandas. After that, the headers for each dataset will be created by utilizing the first row (index 0). Further details may be explained in-line.

For Dataset 1, since some of our visualizations will use data from individual states, while other visualizations use the country as a whole, we need to make sure we can call the API for both datasets.

In [81]:
# Dataset 1: Company Summary
url1_state = requests.get('https://api.census.gov/data/2018/abscs?get=GEO_ID,NAME,STATE,NAICS2017,NAICS2017_LABEL,YIBSZFI,YIBSZFI_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,EMP,EMP_F,PAYANN,PAYANN_F&for=state:*&key=YOUR_API_KEY')
conv1_state = url1_state.json()
df1_state = pd.DataFrame(conv1_state)

df1_state_header = df1_state.iloc[0] #Grab the first row for the header
df1_state = df1_state[1:] #Take the data less the header row
df1_state.columns = df1_state_header #Set the header row as the df header
df1_state = df1_state.astype({'PAYANN': float, 'EMP': float}) #Get payroll and number of employees as number values

# Unhash the line below to view the table:
#df1_state

url1_us = requests.get('https://api.census.gov/data/2018/abscs?get=GEO_ID,NAME,STATE,NAICS2017,NAICS2017_LABEL,YIBSZFI,YIBSZFI_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,EMP,EMP_F,PAYANN,PAYANN_F&for=us:*&key=YOUR_API_KEY')
conv1_us = url1_us.json()
df1_us = pd.DataFrame(conv1_us)

df1_us_header = df1_us.iloc[0] 
df1_us = df1_us[1:] 
df1_us.columns = df1_us_header 
df1_us = df1_us.astype({'PAYANN': float, 'EMP': float})

# Unhash the line below to view the table:
#df1_us

In [82]:
# Dataset 2: Characteristics of Businesses
url2 = requests.get('https://api.census.gov/data/2018/abscb?get=QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,SEX,SEX_LABEL,RCPSZFI,RCPSZFI_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,RCPPDEMP,RCPPDEMP_PCT&for=us&key=YOUR_API_KEY')
conv2 = url2.json()
df2 = pd.DataFrame(conv2)

df2_header = df2.iloc[0]
df2 = df2[1:]
df2.columns = df2_header

# Unhash the line below to view the table:
#df2

In [83]:
# Dataset 3: Characteristics of Business Owners
url3 = requests.get('https://api.census.gov/data/2018/abscbo?get=NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,OWNPDEMP,OWNPDEMP_F,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR&for=us:*&key=YOUR_API_KEY')
conv3 = url3.json()
df3 = pd.DataFrame(conv3)

df3_header = df3.iloc[0]
df3 = df3[1:] 
df3.columns = df3_header

# Unhash the line below to view the table:
#df3

In [77]:
# Dataset 4: Technology Characteristics of Businesses

# Data Transformation
As stated before, the following data transformations are specific to the individual, which means that syntax and methods will vary between each person. Supplemental markdowns and in-line comments will be used to explain the transformations.

# Douglas

Our first set of visualizations aims at looking into how much payroll is dedicated to different racial groups by state and by business category. However, since our data only makes up a survey of businesses in the United States, the raw payroll values in our dataset will probably be misleading. So, not only do we need to filter our data down to get exactly what we want, we also need to mutate our data to get the proper values to analyse.

We also need to be wary of missing data within the dataset. The variables 'PAYROLL_F' and 'EMP_F', which stand for payroll_flag and employee_flag, respectively, are indicator variables to tell us not only whether or not there is missing data, but why. We want the rows with no flag, which indicates there being nothing wrong with our data.

We now need to make sure that the only break out category included within our data is whichever racial group we want to highlight. So, we need to make sure every other category, ethnic group, sex, veteran status, firm size, number of years in business, and business type, needs to be set to include all categories.

In [86]:
df_filtered = df1_state[df1_state['PAYANN_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMP_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMPSZFI'] == '001']
df_filtered = df_filtered[df_filtered['NAICS2017_LABEL'] == 'Total for all sectors']
df_filtered = df_filtered[df_filtered['YIBSZFI_LABEL'] == 'All firms']
df_filtered = df_filtered[df_filtered['SEX'] == '001']
df_filtered = df_filtered[filtered['VET_GROUP'] == '001']
df_filtered = df_filtered[filtered['ETH_GROUP'] == '001']
df_black = df_filtered[df_filtered['RACE_GROUP_LABEL'] == 'Black or African American']

NameError: name 'filtered' is not defined

For the sake of the visualization we want to make, we need to repeat this process, but instead of breaking into race, we want to include all businesses.

In [ ]:
df_total  = df_filtered[df_filtered['RACE_GROUP_LABEL'] == 'Total']

Now, we can select the specific columns we need to build our visualizations from each dataset. The columns we need are 'NAME', 'PAYANN', and 'EMP'.

In [ ]:
df_b = df_black[['NAME', 'PAYANN', 'EMP']]

df_t = df_total[['NAME', 'PAYANN', 'EMP']]

From here, we want to create our new variable: pay per employee. We will do this by dividing the payroll column by the employee column, converting it into a list, and then adding it back into the DataFrame.

In [ ]:
payroll_employee = df_b['PAYANN']/df_b['EMP']

payroll_emp = payroll_employee.tolist()

df_b.insert(3, "Pay per Employee", payroll_employee)

payroll_employee = df_t['PAYANN']/df_t['EMP']

payroll_emp = payroll_employee.tolist()

df_b1.insert(3, "Pay per Employee", payroll_employee)

Finally, we can merge these two datasets together using an inner join on the 'NAME' column.

In [ ]:
df_b_merged = pd.merge(df_b, df_t, how = 'inner', on = 'NAME', suffixes = ('_black', '_total'))

Using this DataFrame, we can build our visualization comparing payroll per employee for black owned businesses with the payroll per employee of businesses overall by state.

Now, we want to do essentially the exact same process, but instead of examining states, we will examine business type. For this, we use 'df1_us' instead of 'df1_state'.

In [ ]:
df_filtered = df1_us[df1_us['PAYANN_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMP_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMPSZFI'] == '001']
df_filtered = df_filtered[df_filtered['NAICS2017_LABEL'] != 'Total for all sectors']
df_filtered = df_filtered[df_filtered['YIBSZFI_LABEL'] == 'All firms']
df_filtered = df_filtered[df_filtered['SEX'] == '001']
df_filtered = df_filtered[filtered['VET_GROUP'] == '001']
df_filtered = df_filtered[filtered['ETH_GROUP'] == '001']
df_black = df_filtered[df_filtered['RACE_GROUP_LABEL'] == 'Black or African American']

In [ ]:
df_total  = df_filtered[df_filtered['RACE_GROUP_LABEL'] == 'Total']

In [ ]:
df_b = df_black[['NAICS2017_LABEL', 'PAYANN', 'EMP']]

df_t = df_total[['NAICS2017_LABEL', 'PAYANN', 'EMP']]

payroll_employee = df_b['PAYANN']/df_b['EMP']

payroll_emp = payroll_employee.tolist()

df_b.insert(3, "Pay per Employee", payroll_employee)

payroll_employee = df_t['PAYANN']/df_t['EMP']

payroll_emp = payroll_employee.tolist()

df_t.insert(3, "Pay per Employee", payroll_employee)

df_b_merged = pd.merge(df_b, df_t, how = 'inner', on = 'NAME', suffixes = ('_black', '_total'))

For our second set of visualizations, we are going to do essentially the same thing, but instead we are going to look at veteran owned businesses. So, we will include all race categories, and compare veteran owned businesses with overall businesses and payroll per employee by state and industry.

In [ ]:
df_filtered = df1_state[df1_state['PAYANN_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMP_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMPSZFI'] == '001']
df_filtered = df_filtered[df_filtered['NAICS2017_LABEL'] == 'Total for all sectors']
df_filtered = df_filtered[df_filtered['YIBSZFI_LABEL'] == 'All firms']
df_filtered = df_filtered[df_filtered['RACE_GROUP_LABEL'] == 'Total']
df_filtered = df_filtered[df_filtered['SEX'] == '001']
df_filtered = df_filtered[filtered['ETH_GROUP'] == '001']
df_veteran = df_filtered[filtered['VET_GROUP'] == '002']

In [ ]:
df_total  = df_filtered[df_filtered['VET_GROUP'] == '001']

In [ ]:
df_v = df_veteran[['NAME', 'PAYANN', 'EMP']]

df_t = df_total[['NAME', 'PAYANN', 'EMP']]

In [ ]:
df_v = df_v[['NAME', 'PAYANN', 'EMP']]

df_t = df_total[['NAME', 'PAYANN', 'EMP']]

payroll_employee = df_v['PAYANN']/df_v['EMP']

payroll_emp = payroll_employee.tolist()

df_b.insert(3, "Pay per Employee", payroll_employee)

payroll_employee = df_t['PAYANN']/df_t['EMP']

payroll_emp = payroll_employee.tolist()

df_t.insert(3, "Pay per Employee", payroll_employee)

df_v_merged = pd.merge(df_v, df_t, how = 'inner', on = 'NAME', suffixes = ('_veteran', '_total'))

And again, we alter the process slightly for the business type as opposed to states.

In [ ]:
df_filtered = df1_us[df1_us['PAYANN_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMP_F'].isna()==True]
df_filtered = df_filtered[df_filtered['EMPSZFI'] == '001']
df_filtered = df_filtered[df_filtered['NAICS2017_LABEL'] != 'Total for all sectors']
df_filtered = df_filtered[df_filtered['YIBSZFI_LABEL'] == 'All firms']
df_filtered = df_filtered[df_filtered['RACE_GROUP_LABEL'] == 'Total']
df_filtered = df_filtered[df_filtered['SEX'] == '001']
df_filtered = df_filtered[filtered['ETH_GROUP'] == '001']
df_veteran = df_filtered[filtered['VET_GROUP'] == '002']

In [ ]:
df_total  = df_filtered[df_filtered['VET_GROUP'] == '001']

In [ ]:
df_v = df_veteran[['NAICS2017_LABEL', 'PAYANN', 'EMP']]

df_t = df_total[['NAICS2017_LABEL', 'PAYANN', 'EMP']]

# Nicole
Nicole sought to answer the following questions:
1. How many businesses are female-owned for each revenue level?
2. What percentage of businesses are owned by non-US citizens and those born outside of the United States?

In Dataset 2, she was examining businesses with only one owner (where 'BUSCHAR' equals 'BQ'), so she had to filter out 'SEX_LABEL' so that there were only 'Male' and 'Female' variables in the column. Including 'Total' and 'Equally male/female' in the data makes no sense for the parameters specified and would skew the results.

Dataset 3 was filtered by birth ('O11') and citizenship ('O12') status. The totals ('EJ' and 'EN') were excluded as well.

In [ ]:
# Dataset 2
filter_sex = df2.loc[(df2['BUSCHAR'] == 'BQ') & (df2['RCPSZFI_LABEL'] != 'All firms') & (df2['SEX_LABEL'] != 'Total') & (df2['SEX_LABEL'] != 'Equally male/female')]

# Dataset 3
filter_born = df3.loc[(df3['QDESC'] == 'O11') & (df3['OWNCHAR'] != 'EJ')]

filter_citizen = df3.loc[(df3['QDESC'] == 'O12') & (df3['OWNCHAR'] != 'EN')]

In [ ]:
# Chris
Chris examined the trends in both education levels and ages of business owners.

In [ ]:
YIB_EMP = df1_us[['YIBSZFI','EMP']] # isolate years-in-business and number-of-employees
YIB_EMP = YIB_EMP.drop(YIB_EMP.index[YIB_EMP['YIBSZFI'] == '001']) # REMOVE specific value in a column where code 001 meant "all firms" which is useless to us
YIB_EMP = YIB_EMP.replace({'YIBSZFI': {'311': '<2', '318': '2-3', '319': '4-5', '321':'6-10','322':'11-15','323':'16+'}}) # replace value codes with values that indicate the year ranges
YIB_EMP['EMP'] = YIB_EMP['EMP'].astype(int) # change EMP type to integer
YIB_EMP = YIB_EMP.rename(columns={'YIBSZFI':'Years In Business','EMP':'Number of Employees'}) # renamed columns
YIB_EMP = YIB_EMP.groupby('Years In Business') # grouped all 'Years In Business' values
YIB_EMP = YIB_EMP.mean() # calculated the average number of employees
YIB_EMP # display 'Years In Business' and 'Number of Employees'

# Owner's Education Level
owner_main = df1_us[['QDESC','QDESC_LABEL','OWNCHAR']] # filter the columns needed
edu = owner_main[(owner_main['QDESC'] == 'O07')] # sort to where QDESC only shows educational section
edu = edu.drop(['QDESC','QDESC_LABEL'], axis=1) # drops to show only the OWNCHAR educational values
edu = edu.drop(edu.index[edu['OWNCHAR'] == 'DS3']) # drops 'Professional degree beyond a bachelor's degree'
edu = edu.drop(edu.index[edu['OWNCHAR'] == 'DT']) # drops 'Total reporting'
edu = edu.drop(edu.index[edu['OWNCHAR'] == 'DU']) # drops 'Item not reported'
edu = edu.replace({'OWNCHAR': {'DM': '>HS', 'DN': 'HS/GED', 'DO': 'Tech/Trade', 'DP':'Some College','DQ':'Associate','DR':'Bachelors','DS1':'Masters','DS2':'Doctorate'}}) # rename generic value codes to actual educational values
edu = edu.rename(columns={'OWNCHAR':'Highest Education Obtained'}) # renamed columns
edu

# Owner's Age
age = owner_main[(owner_main['QDESC'] == 'O09')]  # sort to where QDESC only shows age section
age = age.drop(['QDESC','QDESC_LABEL'], axis=1) # drops to show only the OWNCHAR age values
age = age.drop(age.index[age['OWNCHAR'] == 'EF']) # drops 'Total reporting'
age = age.drop(age.index[age['OWNCHAR'] == 'EG']) # drops 'Item not reported'
age = age.replace({'OWNCHAR': {'DZ': '<25', 'EA': '25-34', 'EB': '35-44', 'EC':'45-54','ED':'55-64','EE':'65+'}}) # rename generic value codes to actual age values
age = age.rename(columns={'OWNCHAR':'Age Range'}) # renamed columns
age

# Dan